# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [70]:
df["Review_len"] = df["Review"].apply(len)
df.head()

,Product,Review,Review_len
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...,492
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi...",176
2,Luxury Air Mattress,This mattress had a small hole in the top of i...,690
3,Pillows Insert,This is the best throw pillow fillers on Amazo...,259
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...,416


## LLMChain

In [ ]:
# !pip install langchain
#!pip install langchain-openai

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [78]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature= 0.9, model="gpt-5-nano")


In [79]:
prompt = ChatPromptTemplate.from_template("Write a review of the product: {input_parameter}?")   

In [80]:

chain = LLMChain(llm=llm, prompt=prompt)

In [81]:
product = df["Product"][0]
chain.run(product)

'Here’s a balanced, generic review for a Queen Size Sheet Set that you can use or adapt if you’re reviewing a specific brand or material.\n\nTitle: Solid value with comfortable everyday use\n\nReview:\nIf you’re shopping on a budget but don’t want to sacrifice comfort, this Queen Size Sheet Set hits a nice middle ground. The feel is soft and breathable, making it pleasant to sleep in, especially in warmer months. The fit on a standard queen mattress (60 x 80) is reliable, and the fitted sheet’s elastic and corner design keep everything in place through the night. The set typically includes a flat sheet and two standard pillowcases, all of which wash well and maintain their shape for multiple spins in the dryer.\n\nMaterial and feel vary by version, but for a cotton-percale option, you’ll enjoy a crisp, matte finish that won’t cling to you. If you prefer a more luxurious, silky-smooth feel, you might opt for a sateen or a higher thread count, as the percale-like options can feel a bit m

## SimpleSequentialChain

In [32]:
from langchain.chains import SimpleSequentialChain

In [82]:
llm = ChatOpenAI(temperature=0.1, model="gpt-5-nano")

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Assess the quality of the product '{product}' based on the review: {review}\n"
    "Provide a rating from 1 to 5 and justify your choice."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="quality_assessment")

In [83]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review of '{product}': {review}\n"
    "Suggest 2-3 specific recommendations for product improvement."
)


# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="improvement_recommendations")

In [85]:
# Prompt 3: Response to the Review
third_prompt = ChatPromptTemplate.from_template(
    "Write a polite, professional response to the customer in the same language as their review for '{product}': {review}\n"
    "Consider the tone of the review and offer a solution to the problems."
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, 
                      output_key="customer_response")   
               

In [86]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["product", "review"],
    output_variables=["quality_assessment", "improvement_recommendations", "customer_response"],
    verbose=True
)

In [ ]:
# Apply the chain to the dataset and save the results
import time

# Initialize new columns
df['quality_assessment'] = ''
df['improvement_recommendations'] = ''
df['customer_response'] = ''

# Apply the chain to each review
for index, row in df.iterrows():
    print(f"Processing review {index + 1}/{len(df)}: {row['Product']}")
    
    try:
        # Run the chain
        result = overall_chain({
            "product": row['Product'],
            "review": row['Review']
        })
        
        # Save results to DataFrame
        df.at[index, 'quality_assessment'] = result['quality_assessment']
        df.at[index, 'improvement_recommendations'] = result['improvement_recommendations']
        df.at[index, 'customer_response'] = result['customer_response']
        
        
    except Exception as e:
        print(f"Error processing review {index}: {e}")
        df.at[index, 'quality_assessment'] = 'Processing Error'
        df.at[index, 'improvement_recommendations'] = 'Processing Error'
        df.at[index, 'customer_response'] = 'Processing Error'

print("Processing complete!")

Processing review 1/7: Queen Size Sheet Set


> Entering new SequentialChain chain...

> Finished chain.
Processing review 2/7: Waterproof Phone Pouch


> Entering new SequentialChain chain...

> Finished chain.
Processing review 3/7: Luxury Air Mattress


> Entering new SequentialChain chain...

> Finished chain.
Processing review 4/7: Pillows Insert


> Entering new SequentialChain chain...

> Finished chain.
Processing review 5/7: Milk Frother Handheld



> Entering new SequentialChain chain...

> Finished chain.
Processing review 6/7: L'Or Espresso Café 



> Entering new SequentialChain chain...

> Finished chain.
Processing review 7/7: Hervidor de Agua Eléctrico


> Entering new SequentialChain chain...

> Finished chain.
Processing complete!


**Repeat the above twice for different products**

In [90]:
# Detailed look at one example
print("Detailed analysis of the first review:")
print("=" * 50)
print(f"Product: {df.iloc[0]['Product']}")
print(f"\nOriginal Review: {df.iloc[0]['Review']}")
print(f"\nQuality Assessment: {df.iloc[0]['quality_assessment']}")
print(f"\nImprovement Recommendations: {df.iloc[0]['improvement_recommendations']}")
print(f"\nCustomer Response: {df.iloc[0]['customer_response']}")

Detailed analysis of the first review:
Product: Queen Size Sheet Set

Original Review: I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.

Quality Assessment: Rating: 4/5

Justification:
- Positive performance: reviewer notes good characteristics and coolness, and is satisfied with the value for the price.
- Service positively impacted the experience: excellent customer service quickly resolved the pillowcase color/matching issue by sending two more matching pillowcases.
- A couple of drawbacks: color mismatch across pillowcase packages and confusion a

In [89]:
# Review the results
print("Updated dataset with analysis results:")
print("=" * 50)
df[['Product', 'Review', 'quality_assessment', 'improvement_recommendations', 'customer_response']].head()

Updated dataset with analysis results:


,Product,Review,quality_assessment,improvement_recommendations,customer_response
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...,Rating: 4/5\n\nJustification:\n- Positive perf...,Here are 3 specific product improvement ideas ...,Thank you for sharing your experience with the...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi...",Rating: 3/5\n\nJustification:\n- Pros: The rev...,Here are 2-3 concrete improvement ideas you co...,"Hello,\n\nThank you for sharing your experienc..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...,Rating: 1/5\n\nJustification:\n- Product defec...,"Here are 3 concrete, product-focused improveme...","Hi there,\n\nThank you for taking the time to ..."
3,Pillows Insert,This is the best throw pillow fillers on Amazo...,Rating: 5/5\n\nJustification: The review is st...,Here are 2–3 concrete improvement ideas based ...,Thank you so much for your kind words! We’re t...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...,Rating: 2 out of 5\n\nJustification:\n- The pr...,Here are 3 concrete ideas to improve the Milk ...,I'm really sorry to hear you’ve had this repea...


## SequentialChain

In [37]:
from langchain.chains import SequentialChain

In [38]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to German:\n\nReview: {review}"
)


chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="german_review" # Даем имя выходу
                    )


In [41]:

second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1-2 sentences:\n\nReview: {german_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" # give a name to this output
                    )


In [44]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
      "Translate the following review to Russian:\n\nReview: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="russian_review"
                      )


In [47]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables

fourth_prompt = ChatPromptTemplate.from_template(
    "Here is the original review in Russian:\n{russian_review}\n\nHere is its summary in German:\n{summary}\n\nWrite a follow-up message to the customer in Russian, addressing their review and summary."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [50]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["summary", "russian_review", "followup_message"],
    verbose=True
)

In [51]:
review = df.Review[5]
overall_chain(review)

/var/folders/hg/7_xt468j3nz1xqspwcc6s8jw0000gn/T/ipykernel_40466/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'summary': 'Der Kunde findet den Geschmack schlecht, bemerkt, dass der Schaum nicht gut hält und vermutet, dass es sich um eine alte Charge oder eine Fälschung handelt im Vergleich zu denen, die er im Laden kauft.',
 'russian_review': 'Отзыв: Я нахожу вкус отвратительным. Пена не держится, это странно. Я покупаю те же самые в магазине, и вкус намного лучше... Старая партия или подделка!?',
 'followup_message': 'Уважаемый клиент,\n\nМы приносим извинения за ваше негативное впечатление от продукции. Мы серьезно относимся к качеству нашей продукции и стремимся обеспечить клиентов только лучшими продуктами.\n\nМы хотели бы уточнить, что наша продукция проходит строгий контроль качества, и мы не используем старые партии или подделки. Возможно, у нас произошла непредвиденная ситуация, и мы готовы рассмотреть вашу пр

## Router Chain

In [91]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [104]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [105]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [106]:
llm = ChatOpenAI(temperature=0)


In [107]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [108]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [109]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [110]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [111]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [100]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation at all wavelengths. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is a function of its temperature and the wavelength of the radiation."

In [101]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [102]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential for the pro

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [112]:
chain.run("Which country had the lowest amount of wars in the last 300 years?")



> Entering new MultiPromptChain chain...
history: {'input': 'Which country had the lowest amount of wars in the last 300 years?'}
> Finished chain.


'It is difficult to determine definitively which country has had the lowest amount of wars in the last 300 years as the number of wars can vary depending on how they are defined and recorded. However, some countries that are often cited as having relatively low levels of conflict in recent history include Switzerland, Sweden, and Costa Rica. These countries have a history of neutrality and have not been involved in many major wars in the past few centuries.'